# Loading a CITE Library



## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [ ]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [ ]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [ ]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source


## Useful Functions

Save a string to a names file:

In [ ]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

Convert a Roman Numeral to an Integer:

In [ ]:
def fromRoman(s: String) : Int = {
	try {
		val numerals = Map('I' -> 1, 'V' -> 5, 'X' -> 10, 'L' -> 50, 'C' -> 100, 'D' -> 500, 'M' -> 1000)

		s.toUpperCase.map(numerals).foldLeft((0,0)) {
		  case ((sum, last), curr) =>  (sum + curr + (if (last < curr) -2*last else 0), curr) }._1
	} catch {
		case e:Exception => throw new Exception(s""" "${s}" is not a valid Roman Numeral.""")
	}
}

## Assemble a Library

Build a CITE Library by aggregating files, to incorporate ongoing editing.

## Load the New Library

Load a CITE Library…

In [ ]:
val cexPath = "https://raw.githubusercontent.com/Furman-Editions-In-Progress/xolotl/master/cex/xolotl_all.cex"
val lib = CiteLibrary(scala.io.Source.fromURL(cexPath).mkString)

### Get the Components of the Library

**Warning** The code below makes some assumptions about this particular CiteLibrary, namely that it has a TextRepository and a CollectionRepository. Those `.get` methods will throw an exception in the even that the loaded library has `None` as the value for `.textRepository` or `.collectionRepository`.

In [ ]:
val tr = lib.textRepository.get
val cat = tr.catalog
val corp = tr.corpus
val cr = lib.collectionRepository.get

## Find Things in a Scene

Get the URN for a scene we are interested in.

In [ ]:
val sceneComment = Cite2Urn("urn:cite2:xolotl:commentary.v1:1")
val sceneImgProperty = cr.data ~~ Cite2Urn("urn:cite2:xolotl:commentary.v1.img:1")
val sceneImg = sceneImgProperty.data.head.propertyValue
val sceneImgROI = sceneImg.asInstanceOf[Cite2Urn].objectExtensionOption
val sceneRoi = ImageROI(sceneImgROI.get)